In [1]:
import sys
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import time
import pytz
import pandas as pd
import json 
import re


In [2]:
url='https://speakday.com/timetable/english-conversations'


In [3]:
def test_print(url):
    print(url)

In [4]:
def get_page(url):  #be careful, we don't want to crash the site
    HEADERS = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Cache-Control": "max-age=0",
        }
    html = requests.get(url, headers=HEADERS)
    bs = BeautifulSoup(html.text, 'html.parser')
    return  bs

In [9]:


#filter data
def filter_speakday(bs):  
    script_json=bs.find('script', id='__NEXT_DATA__')
    json_object = json.loads(script_json.contents[0])
    def fun_df_week(week): #current or next_week
        list_seesion_big=json_object['props']['pageProps']['timetableData']['conversations'][week]
        list_seesion=[]
        for session in list_seesion_big:
            for i in session:
                if i in ('schedule'):
                    schedule =(i,session[i]['date_time'])
            dict_small=dict([schedule])
            list_seesion.append(dict_small)
        df_import_week = pd.DataFrame.from_records(list_seesion)
        return df_import_week

    
    df_import_next_week=fun_df_week('next_week')
    df_import_current_week=fun_df_week('current')

    df_import = pd.concat([df_import_next_week, df_import_current_week])
    df_import['schedule']=pd.to_datetime(df_import['schedule']).dt.tz_convert('Europe/Kyiv')
    
    #get current dateTime
    current_dateTime = pd.to_datetime('today').tz_localize('Europe/Kyiv')
    #end time session only 20 minutes
    time_change = timedelta(minutes=20)



    #filter by upcoming events only
    df_not_group= df_import[df_import['schedule'] >= current_dateTime].copy()



    #grouping
    df_not_group = df_not_group.rename(columns={'schedule': 'start_event'})
    df_not_group['end_event']=df_not_group['start_event']+time_change
    
    df_not_group=df_not_group.sort_values(by='start_event')
    df_upcoming = df_not_group.groupby(
        (df_not_group["end_event"].diff() != pd.Timedelta("20 minutes")).cumsum(), as_index=False
        ).agg({"start_event": "first", "end_event": "last"})
    
    

    #change time fromat
    df_upcoming["start_event"] = df_upcoming["start_event"].apply(lambda x: x.isoformat())
    df_upcoming["end_event"] = df_upcoming["end_event"].apply(lambda x: x.isoformat())
    
    #final table
    df_upcoming['event_name']='Speakday'
    df_upcoming['description']=url
    df_upcoming['location']=None
    df_upcoming=df_upcoming[['event_name', 'description',  'start_event', 'end_event', 'location']]

    
    return df_upcoming



In [10]:
#be careful, we don't want to crash the site
#bs_speakday=get_page(url)

In [11]:
#bs_speakday

In [12]:
df_upcoming_speakday=filter_speakday(bs_speakday)

In [13]:
df_upcoming_speakday

,event_name,description,start_event,end_event,location
0,Speakday,https://speakday.com/timetable/english-convers...,2023-09-28T21:00:00+03:00,2023-09-28T22:00:00+03:00,None
1,Speakday,https://speakday.com/timetable/english-convers...,2023-09-30T17:00:00+03:00,2023-09-30T18:00:00+03:00,None
2,Speakday,https://speakday.com/timetable/english-convers...,2023-10-01T17:00:00+03:00,2023-10-01T18:00:00+03:00,None
3,Speakday,https://speakday.com/timetable/english-convers...,2023-10-02T21:00:00+03:00,2023-10-02T22:00:00+03:00,None
4,Speakday,https://speakday.com/timetable/english-convers...,2023-10-03T19:00:00+03:00,2023-10-03T20:00:00+03:00,None
5,Speakday,https://speakday.com/timetable/english-convers...,2023-10-04T05:00:00+03:00,2023-10-04T06:00:00+03:00,None
6,Speakday,https://speakday.com/timetable/english-convers...,2023-10-05T21:00:00+03:00,2023-10-05T22:00:00+03:00,None
7,Speakday,https://speakday.com/timetable/english-convers...,2023-10-07T17:00:00+03:00,2023-10-07T18:00:00+03:00,None
8,Speakday,https://speakday.com/timetable/english-convers...,2023-10-08T17:00:00+03:00,2023-10-08T18:00:00+03:00,None
